
<img style="float: right;" src="KOA.png" width =500 height= 500 />


# <font color="#0b5394"> __Interactive Visualization of Keck Observations of Protostars in the $\rho$ Ophiuchus Dark Cloud__

## <font color="#0b5394"> __John C. Good, Rosemary Moseley, G Bruce Berriman, Christopher R. Gelino (Caltech/IPAC-NExScI)__

### This Notebook retrieves the metadata for observations acquired at the Keck Observatory of sources in the $\rho$ Oph Cloud measured at the Keck Observatory. The observations are overlaid on a Spitzer IRAC image, accessed from IRSA, of the $\rho$ Oph region, along with the positions of a table of protostars downloaded from Vizier. We describe the steps in detail as an aid to editing the Notebook for other use cases. 

### The Notebook takes advantage of the Keck Observatory Archive (KOA)'s Data Discovery Service. In a few seconds,  it captures an inventory of all public observations of the $\rho$ Oph Cloud at KOA. KOA curates all data acquired at the Keck Observatory since it began operating in 1994. 
    
### The steps are:

#### - Look up the position of Rho Oph in Simbad and define a search radius.
#### - Find out which Keck Instruments have made observations in this region through an inventory search of the Keck Observatory Archive.
#### - Look up a list of the Protostars in $\rho$ Oph from Vizier.
#### - Download from IRSA a Spitzer IRAC mosaic of $\rho$ Oph 
#### - Define the layout of the visualization display, and finally
#### - Create the interactive visualization



### <font color="#0b5394"> Requirements
    
Please install these packages:
    
pip install plotly   

pip install dash_bootstrap_components

pip install dash_ag_grid

pip install mviewer

pip install koaviewer

pip install astroquery

pip install MontagePy

This notebook has been tested on Python 3.10, 3.11, 3.12, and 3.13


## <font color="#0b5394"> Set up
    
Import required packages and define a utility function.



In [88]:
# Always run this cell.

import os
import shutil
import MontagePy
import urllib.request
from astropy.io import ascii
from MontagePy.main import mShrink


# General function for retrieving URL to file

def retrieve_url_to_file(url, filename):
    
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"Successfully downloaded {url} to {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")

## <font color="#0b5394"> Region Parameters and Creating Working Space
By default, working space is set up as a subdirectory of '~/data'.  Change the code below if you want to use a different location.

In [89]:
object    = 'rho Oph'
radius    = 1.5
workspace = ('~/data/' + object).replace(' ', '_')

workspace = os.path.expanduser(workspace)

# Uncomment the next few of lines if you want this script to automatically
# remove any previous data under this object name in the workspace.

#try:
#    shutil.rmtree(workspace)
#except:
#    pass


In [90]:
try:
    if os.path.isdir(workspace) or os.path.isfile(workspace):
        print('ERROR: ' + workspace + ' already exists.  Delete before continuing.')
    else:
        os.makedirs(workspace + '/tbl')
        os.makedirs(workspace + '/fits')
        
        print('Created ' + workspace)
except:
    print('ERROR creating workspace: ' + workspace)
    

Created /Users/gbb/data/rho_Oph


## <font color="#0b5394"> Object Lookup
Look up the position of Rho Oph in Simbad and define a search radius. This example uses a radius of 0.5 degrees. Define a workspace to store query results.

In [91]:
from astroquery.simbad import Simbad

result = Simbad.query_object(object)

ra  = result[0]['ra']
dec = result[0]['dec']

print('object:    ', object)
print('region:    ', ra, dec, radius)
print('workspace: ', workspace)

object:     rho Oph
region:     246.39632358927884 -23.44717083027709 1.5
workspace:  /Users/gbb/data/rho_Oph


## <font color="#0b5394"> Search the Keck Observatory Archive holdings.

Retrieves the metadata of all observations archived at KOA

The KOA data discovery service is accessed directly via URL, and requires ra, dec and radius in degrees. It also supports a time range, not used here).  The results are written to the workspace as IPAC column delimited files and as CSV files. In this example, eleven Keck instruments have made observations of $\rho$ Oph

In [92]:

tblfile   = 'counts.tbl'

ftbl = workspace + '/tbl/' + tblfile

urlbase = 'https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?'
urlbase = urlbase + 'ra=' + str(ra) + '&dec=' + str(dec) 
urlbase = urlbase + '&radius=' + str(radius)
urlbase = urlbase + '&tmin=-1.e9&tmax=1.e9'

retrieve_url_to_file(urlbase, ftbl)

data = ascii.read(ftbl)

print(data)

for row in data:
    catalog = row['identifier'].lower()
    
    tblfile = catalog + '.tbl'

    ftbl = workspace + '/tbl/' + tblfile 

    url = urlbase + '&catalog=' + catalog

    retrieve_url_to_file(url, ftbl)


Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=1.5&tmin=-1.e9&tmax=1.e9 to /Users/gbb/data/rho_Oph/tbl/counts.tbl
identifier count
---------- -----
    DEIMOS     7
       ESI     2
     HIRES   150
      LRIS   549
       LWS   195
   MOSFIRE    61
     NIRC2 12824
      NIRC  1933
     NIRES   124
   NIRSPEC 21809
    OSIRIS  1017
    GUIDER  1384
Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=1.5&tmin=-1.e9&tmax=1.e9&catalog=deimos to /Users/gbb/data/rho_Oph/tbl/deimos.tbl
Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=1.5&tmin=-1.e9&tmax=1.e9&catalog=esi to /Users/gbb/data/rho_Oph/tbl/esi.tbl
Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=1.5&t

### <font color="#0b5394"> Verify tables are in output directory

In [93]:
import subprocess

result = subprocess.run(["ls", "-R", workspace + "/tbl/"], check=True, capture_output=True, text=True)
print("Command output:")
print(result.stdout)

Command output:
counts.tbl
deimos.tbl
esi.tbl
guider.tbl
hires.tbl
lris.tbl
lws.tbl
mosfire.tbl
nirc.tbl
nirc2.tbl
nires.tbl
nirspec.tbl
osiris.tbl



<img style="float: right;" src="https://content.cld.iop.org/journals/0004-637X/835/1/3/revision1/apjaa51d4f2_lr.jpg" width =300 height= 300 />


## <font color="#0b5394"> Acquire a list of the Protostars in $\rho$ Oph

Call Vizer's  VO-compliant Table Access Protocol (TAP) server to download the table of 38 protostars in Table 3 of "218GHz obs. of embedded protostars in Ophiuchus". Johan E. Lindberg et al (2017) ApJ 835, 3. (J/ApJ/835/3).  https://iopscience.iop.org/article/10.3847/1538-4357/835/1/3See also the Vizier page at https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJ/835/3
    
The figure on the right shows the locations of these sources (Figure 2 in Lindberg et al. The red circles denote the postiions of them stars in the study, and the blue ring is the luminous source S1.


In [94]:
from astropy.io import ascii

csvfile   = 'protostars.csv'
tblfile   = 'protostars.tbl'

fcsv = workspace + '/tbl/' + csvfile
ftbl = workspace + '/tbl/' + tblfile

url = 'http://tapvizier.u-strasbg.fr/TAPVizieR/tap/sync?'
url = url + 'request=doquery&lang=adql&query='
url = url + 'select+*+from+"J/ApJ/835/3/table1"'
url = url + "&format=csv"

retrieve_url_to_file(url, fcsv)


# Convert VizieR CSV format to IPAC table

data = ascii.read(fcsv)
data.rename_column('RAJ2000', 'ra')
data.rename_column('DEJ2000', 'dec')
data.write(ftbl, format="ipac", overwrite=True)

print('Converted to', ftbl)

Successfully downloaded http://tapvizier.u-strasbg.fr/TAPVizieR/tap/sync?request=doquery&lang=adql&query=select+*+from+"J/ApJ/835/3/table1"&format=csv to /Users/gbb/data/rho_Oph/tbl/protostars.csv
Converted to /Users/gbb/data/rho_Oph/tbl/protostars.tbl


## <font color="#0b5394">Download a Spitzer IRAC mosaic of $\rho$ Oph

Be sure to wait for the download to finish.


In [95]:
file = 'rhoOph_large.fits'

fimg = workspace + '/fits/' + file

url = 'https://irsa.ipac.caltech.edu/data/SPITZER/Enhanced/SEIP/images/'
url = url + '4/0015/40015561/7/40015561-87/40015561.40015561-87.IRAC.1.mosaic.fits'

retrieve_url_to_file(url, fimg)


Successfully downloaded https://irsa.ipac.caltech.edu/data/SPITZER/Enhanced/SEIP/images/4/0015/40015561/7/40015561-87/40015561.40015561-87.IRAC.1.mosaic.fits to /Users/gbb/data/rho_Oph/fits/rhoOph_large.fits


### <font color="#0b5394"> Shrink the Background Image

It is often useful to shrink a large image for fast presentation.  In our case, the image size is 4000 x 4000 pixel, and we shrink it in both dimensions by a factor of four.

In [96]:
infile        = 'rhoOph_large.fits'
outfile       = 'rhoOph.fits'
shrink_factor = 4

fin  = workspace + '/fits/' + infile
fout = workspace + '/fits/' + outfile

mShrink(fin, fout, shrink_factor)

{'status': '0', 'time': 0.0}

### <font color="#0b5394"> Verify files are written to the output directory

In [97]:
import subprocess

result = subprocess.run(["ls", "-R", workspace], check=True, capture_output=True, text=True)
print("Command output:")
print(result.stdout)


Command output:
fits
tbl

/Users/gbb/data/rho_Oph/fits:
rhoOph.fits
rhoOph_large.fits

/Users/gbb/data/rho_Oph/tbl:
counts.tbl
deimos.tbl
esi.tbl
guider.tbl
hires.tbl
lris.tbl
lws.tbl
mosfire.tbl
nirc.tbl
nirc2.tbl
nires.tbl
nirspec.tbl
osiris.tbl
protostars.csv
protostars.tbl



###  <font color="#0b5394"> __Define the layout of the visualization display__

__Please read this description before running the next two cells__

The 'koaViewer' app helps you visualize the data you have downloaded into the workspace.  This workspace contains a 'tbl' subdirectory with all the table files downloaded (source tables and image metadata tables) and a 'fits' directory with a FITS image file(s).  We need to specify which tables to display, the colors of overlays in the image display, symbol sizes, <i>etc.</i>
The parameters of the image display is defined by a JSON structure file ('layout.json') in the top level of the workspace.  

There are three ways to build the JSON file.  
- If no layout.json exists when koaViewer starts up, it will analyze the contents of the workspace and take a guess at the structure using a few simple rules.  This is likely to be sub-optimal.
-   Alternatively, knowing what we have done in the Notebook to construct the workspace, we can construct the JSON manually (<i>i.e.,</i> setting up the JSON in a Notebook cell).
-   Finally, once we have a starting point, the koaViewer application has layout editing pop-ups where we can fine tune the details (such as  color and column-based symbol sizes).

We define the JSON structure in the next cell.

In [98]:
import json


# Default image/overlay layout

layout = {
  "Image Label": "rho Oph: Spitzer IRAC",

  "Grayscale":
  [
    {
      "File": "fits/rhoOph.fits",
      "Color Table": "Grayscale",
      "Stretch Mode": "Gaussian-log",
      "Min": "-1.0", "Min Units": "sigma",
      "Max": "max", "Max Units": "sigma"
    }
  ],

  "Overlays":
  [
    {"Type": "Catalog", "Color": "ffffff", "File": "tbl/deimos.tbl",     "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ff8080", "File": "tbl/hires.tbl",      "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "8080ff", "File": "tbl/lris.tbl",       "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffff80", "File": "tbl/lws.tbl",        "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "80ffff", "File": "tbl/nirc.tbl",       "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ccccff", "File": "tbl/nirc2.tbl",      "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffbf00", "File": "tbl/osiris.tbl",     "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "80ff80", "File": "tbl/esi.tbl",        "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ff80ff", "File": "tbl/nires.tbl",      "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffaaaa", "File": "tbl/nirspec.tbl",    "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffffff", "File": "tbl/protostars.tbl", "Size of Ref Value":  5.0, "Symbol": "circle"},
    {"Type": "Eq grid", "Color": "808000"}
  ]
}


# Special check: remove any overlays where we failed to find any catalog data

overlays = layout["Overlays"]

for overlay in overlays:
    if overlay["Type"] == "Catalog":
        file = workspace + "/" + overlay["File"]
        if not os.path.isfile(file):
            overlays.remove(overlay)
        
layout["Overlays"] = overlays


# Save the updated layout to a JSON file

filename = os.path.expanduser(workspace) + "/layout.json"

with open(filename, 'w') as file:
    json.dump(layout, file, indent=2)

print("JSON layout written to", filename)


JSON layout written to /Users/gbb/data/rho_Oph/layout.json


## Create the interactive visualization

To start koaViewer, run the next cell and activate the link it produces to examine the visualization. 

In [99]:
from koaviewer import koaViewer

viewer = koaViewer()

viewer.run(workspace)

Dash app running on http://127.0.0.1:51655/


### <font color= "#0b5394">Acknowledgements
The  Keck Observatory Archive (KOA) is operated by the W. M. Keck Observatory and the NASA Exoplanet Science Institute (NExScI), Caltech, under contract with the National Aeronautics and Space Administration.

Montage has been funded by the National Science Foundation under Grant Numbers ACI-1440620,1642453 and 1835379, and was previously funded by the National Aeronautics and Space Administration's Earth Science Technology Office, Computation Technologies Project, under Cooperative Agreement Number NCC5-626 between NASA and the California Institute of Technology. 

### <font color="#0b5394"> Need help?

Contact the KOA help desk at https://koa.ipac.caltech.edu/cgi-bin/Helpdesk/nph-genTicketForm?projname=KOA 

##  <p style="text-align:center;"> <font color="#0b5394">  Visit the Keck Observatory Archive at https://koa.ipac.caltech.edu/ </p>